<a href="https://colab.research.google.com/github/jhjh5823/Crawling/blob/master/%EC%A7%80%EB%8B%88%EC%B0%A8%ED%8A%B8Top100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

지니 Top 200차트

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)

In [3]:
#Chrome User-Agent
header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
req = requests.get(url, headers=header)
soup = BeautifulSoup(req.text)

# 2. 찾으려고 하는 데이터의 도드 찾기

In [4]:
trs = soup.select('tr.list')

In [5]:
#개발자 도구에서 찾아주는 selector
trs = soup.select('#body-content > div.newest-list > div > table > tbody > tr')
len(trs)
#body-content > div.newest-list > div

50

In [6]:
#body-content > div.newest-list > div > table > tbody

In [22]:
tr = trs[12]

In [8]:
#rank
tr.select_one('.number').get_text()

'13\n                                        \n                                    \n유지\n\n'

In [9]:
try:
  rank = int(tr.select_one('.number').get_text().split('\n')[0])
except:
  rank=0

In [10]:
rank

13

In [11]:
#제목
try : #일반적인 노래
  title = tr.select_one('.title.ellipsis').string.strip()
except: #19금 노래
  title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
title

'Yet To Come'

In [12]:
#artist
artist = tr.select_one('.artist.ellipsis').text.strip()
artist

'방탄소년단'

In [13]:
#album
album = tr.select_one('.albumtitle.ellipsis').text.strip()
album

'Proof'

# 4. 한페이지 내에서 있는 정보 가져오기

In [14]:
rank_list, title_list, artist_list, album_list = [], [], [], []

for i, tr in enumerate(trs):
  try:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
  except:
    rank=0

  try : #일반적인 노래
    title = tr.select_one('.title.ellipsis').string.strip()
    

  except: #19금 노래
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()
    
  artist = tr.select_one('.artist.ellipsis').text.strip()
  album = tr.select_one('.albumtitle.ellipsis').text.strip()
  rank_list.append(rank)
  title_list.append(title)
  artist_list.append(artist)
  album_list.append(album)


In [16]:
lines = []

for i, tr in enumerate(trs):
  try:
    rank = int(tr.select_one('.number').get_text().split('\n')[0])
  except:
    rank=0

  try : #일반적인 노래
    title = tr.select_one('.title.ellipsis').string.strip()
    
  except: #19금 노래
    title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()  
  artist = tr.select_one('.artist.ellipsis').text.strip()
  album = tr.select_one('.albumtitle.ellipsis').text.strip()
  line={'순위':rank, '제목':title, '아티스트':artist, '앨범':album}
  lines.append(line)

# 5. 데이터프레임 만들기

In [23]:
#df = pd.DataFrame(lines)
df = pd.DataFrame({'순위' : rank_list, '제목':title_list, '아티스트':artist_list, '앨범':album_list})

# 6. 모든 페이지에 대해 가져오기

In [19]:
lines = []
for page in range(1, 5):
  url =f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20220502&hh=15&rtm=Y&pg={page}'
  header = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.127 Safari/537.36'}
  req = requests.get(url, headers=header)
  soup = BeautifulSoup(req.text)
  trs = soup.select('tr.list')

  for i, tr in enumerate(trs):
    try:
      rank = int(tr.select_one('.number').get_text().split('\n')[0])
    except:
      rank=0

    try : #일반적인 노래
      title = tr.select_one('.title.ellipsis').string.strip()
      
    except: #19금 노래
      title = tr.select_one('.title.ellipsis').get_text().split('\n')[-1].strip()  
    artist = tr.select_one('.artist.ellipsis').text.strip()
    album = tr.select_one('.albumtitle.ellipsis').text.strip()
    line={'순위':rank, '제목':title, '아티스트':artist, '앨범':album}
    lines.append(line)


In [20]:
df3 = pd.DataFrame(lines)
df3.tail(4)

,순위,제목,아티스트,앨범
196,197,DM,프로미스나인,Midnight Guest
197,198,Kiss Me More (Feat. SZA),Doja Cat,Kiss Me More
198,199,사랑의 인사,씨야 (SeeYa),Lovely Sweet Heart
199,200,흰눈,먼데이 키즈 (Monday Kiz),흰눈


In [21]:
df3.to_csv('지니차트 Top200.csv',index=False)